# Question 1 - NHANES Table 1 [35 points]

## part b) 
The variable OHDDESTS contains the status of the oral health exam. Merge this variable into the demographics data.

Use the revised demographic data from part a and the oral health data from PS2 to create a clean dataset with the following variables:

- id (from SEQN)
- gender
- age
- under_20 if age < 20
- college - with two levels:
    - ‘some college/college graduate’ or
    - ‘No college/<20’ where the latter category includes everyone under 20 years of age.
- exam_status (RIDSTATR)
- ohx_status - (OHDDESTS)//dentition_status	

In [251]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy import stats
from IPython.core.display import display, HTML
import feather
from os.path import exists

In [252]:
filepath = "/Users/ShuyanLi/desktop/Umich_lsy/STATS507/HW4"

In [253]:
# read data files
if exists((filepath+"/demo.feather") and (filepath+"/ohx.feather")):
    demo = pd.read_feather(filepath+"/demo.feather")
    ohx = pd.read_feather(filepath+"/ohx.feather")
else:
    print("There is no such file!")
demo

,id,age,gender,race,education,marital_status,exam_status,psu,strata,exam_wt,interview_wt,cohort
0,62161,22,Male,Non-Hispanic White,High school graduate/GED or equivalent,Never married,Both interviewed and MEC examined,1,91,104236.582554,102641.406474,2011-2012
1,62162,3,Female,Mexican American,NaN,NaN,Both interviewed and MEC examined,3,92,16116.354010,15457.736897,2011-2012
2,62163,14,Male,Non-Hispanic Asian,NaN,NaN,Both interviewed and MEC examined,3,90,7869.485117,7397.684828,2011-2012
3,62164,44,Female,Non-Hispanic White,Some college or AA degree,Married,Both interviewed and MEC examined,1,94,127965.226204,127351.373299,2011-2012
4,62165,14,Female,Non-Hispanic Black,NaN,NaN,Both interviewed and MEC examined,2,90,13384.042162,12209.744980,2011-2012
...,...,...,...,...,...,...,...,...,...,...,...,...
39151,102952,70,Female,Non-Hispanic Asian,High school graduate/GED or equivalent,Married,Both interviewed and MEC examined,2,138,18338.711104,16896.276203,2017-2018
39152,102953,42,Male,Mexican American,High school graduate/GED or equivalent,Separated,Both interviewed and MEC examined,2,137,63661.951573,61630.380013,2017-2018
39153,102954,41,Female,Non-Hispanic Black,College graduate or above,Never married,Both interviewed and MEC examined,1,144,17694.783346,17160.895269,2017-2018
39154,102955,14,Female,Non-Hispanic Black,NaN,NaN,Both interviewed and MEC examined,1,136,14871.839636,14238.445922,2017-2018


In [254]:
ohx

,id,dentition_status,tc_01,tc_02,tc_03,tc_04,tc_05,tc_06,tc_07,tc_08,...,ctc_23,ctc_24,ctc_25,ctc_26,ctc_27,ctc_28,ctc_29,ctc_30,ctc_31,cohort
0,62161,Complete,Tooth not present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,...,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Permanent tooth with a dental carious surface ...,Sound permanent tooth,2011-2012
1,62162,Complete,Tooth not present,Tooth not present,Tooth not present,Primary tooth present,Primary tooth present,Primary tooth present,Primary tooth present,Primary tooth present,...,Sound primary tooth,Sound primary tooth,Sound primary tooth,Sound primary tooth,Sound primary tooth,Sound primary tooth,Sound primary tooth,Unerupted,Unerupted,2011-2012
2,62163,Complete,Tooth not present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,...,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,"Tooth present, condition cannot be assessed",Sound permanent tooth,2011-2012
3,62164,Complete,Tooth not present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,...,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Permanent tooth with a dental carious surface ...,Permanent tooth with a dental carious surface ...,2011-2012
4,62165,Complete,Tooth not present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,...,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,2011-2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35904,102952,Complete,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,...,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,2017-2018
35905,102953,Complete,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,...,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Permanent tooth with a dental carious surface ...,2017-2018
35906,102954,Complete,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,...,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Permanent tooth with a restored surface condition,Sound permanent tooth,Sound permanent tooth,2017-2018
35907,102955,Complete,Tooth not present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,Permanent tooth present,...,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound permanent tooth,Sound perman

In [255]:
#extact "id", "dentition_status", "cohort" from ohx dataframe
ohx_mini = ohx[["id", "dentition_status", "cohort"]]
# change "dentition_status" column name into "ohx_status"
ohx_mini.columns = ["id", "ohx_status", "cohort"]

In [256]:
# extract "id", "gender", "age", "exam_status", "education" from demo dataframe
demo_mini = demo[["id", "gender", "age", "exam_status", "education", "cohort"]]
# Add a column "under_20" 
demo_mini["under_20"] = demo_mini.apply(lambda x: "Yes" if x['age']<20 else "No", axis=1)

/Users/ShuyanLi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [257]:
# Add column "college"
def function(edu, age):
    """
    Define the category of "college": 
    "some college/college graduate" or "No college/<20".

    Parameters
    ----------
    edu : string
        The string representing the education level. 
    age : int
        Values of age. 

    Returns
    -------
    The category of "college": 
    "some college/college graduate" or "No college/<20". 

    """
    if ((edu=="Some college or AA degree" or edu=="College graduate or above") and age>=20):
        return ("some college/college graduate")
    else:
        return ("No college/<20")

demo_mini["college"] = demo_mini.apply(lambda x: function(x["education"], x["age"]), axis=1)

/Users/ShuyanLi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [258]:
mergedata_b = pd.merge(demo_mini[["id", "gender", "age", "under_20", "college", "exam_status", "cohort"]], ohx_mini, on=["id", "cohort"], how="left")

In [259]:
mergedata_b = mergedata_b[["id", "gender", "age", "under_20", "college", "exam_status", "ohx_status", "cohort"]]
mergedata_b

,id,gender,age,under_20,college,exam_status,ohx_status,cohort
0,62161,Male,22,No,No college/<20,Both interviewed and MEC examined,Complete,2011-2012
1,62162,Female,3,Yes,No college/<20,Both interviewed and MEC examined,Complete,2011-2012
2,62163,Male,14,Yes,No college/<20,Both interviewed and MEC examined,Complete,2011-2012
3,62164,Female,44,No,some college/college graduate,Both interviewed and MEC examined,Complete,2011-2012
4,62165,Female,14,Yes,No college/<20,Both interviewed and MEC examined,Complete,2011-2012
...,...,...,...,...,...,...,...,...
39151,102952,Female,70,No,No college/<20,Both interviewed and MEC examined,Complete,2017-2018
39152,102953,Male,42,No,No college/<20,Both interviewed and MEC examined,Complete,2017-2018
39153,102954,Female,41,No,some college/college graduate,Both interviewed and MEC examined,Complete,2017-2018
39154,102955,Female,14,Yes,No college/<20,Both interviewed and MEC examined,Complete,2017-2018


## part c)
Remove rows from individuals with exam_status != 2 as this form of missingness is already accounted for in the survey weights. Report the number of subjects removed and the number remaining.

In [260]:
data_c = mergedata_b.drop(index=(mergedata_b.loc[(mergedata_b['exam_status']!='Both interviewed and MEC examined')].index))
data_c

,id,gender,age,under_20,college,exam_status,ohx_status,cohort
0,62161,Male,22,No,No college/<20,Both interviewed and MEC examined,Complete,2011-2012
1,62162,Female,3,Yes,No college/<20,Both interviewed and MEC examined,Complete,2011-2012
2,62163,Male,14,Yes,No college/<20,Both interviewed and MEC examined,Complete,2011-2012
3,62164,Female,44,No,some college/college graduate,Both interviewed and MEC examined,Complete,2011-2012
4,62165,Female,14,Yes,No college/<20,Both interviewed and MEC examined,Complete,2011-2012
...,...,...,...,...,...,...,...,...
39151,102952,Female,70,No,No college/<20,Both interviewed and MEC examined,Complete,2017-2018
39152,102953,Male,42,No,No college/<20,Both interviewed and MEC examined,Complete,2017-2018
39153,102954,Female,41,No,some college/college graduate,Both interviewed and MEC examined,Complete,2017-2018
39154,102955,Female,14,Yes,No college/<20,Both interviewed and MEC examined,Complete,2017-2018


The number of subjects removed = 39156 - 37399 = 1757

The number remaining = 37399

## part d)
Construct a table with ohx (complete / missing) in columns and each of the following variables summarized in rows:

- age
- under_20
- gender
- college

For the rows corresponding to categorical variable in your table, each cell should provide a count (n) and a percent (of the row) as a nicely formatted string. For the continous variable age, report the mean and standard deviation [Mean (SD)] for each cell.

Include a column ‘p-value’ giving a p-value testing for a mean difference in age or an association beween each categorical varaible and missingness. Use a chi-squared test comparing the 2 x 2 tables for each categorical characteristic and OHX exam status and a t-test for the difference in age.

**Hint*: Use scipy.stats for the tests.

In [261]:
## For college
data_college = data_c
# modify "ohx_status" as "complete" or "missing"
data_college["ohx status"] = data_c.apply(lambda x: "complete" if x["ohx_status"]=="Complete" else "missing", axis=1)
ohxstatus_college = data_college.groupby(["college","ohx status"]).agg({"ohx status":"count"})
ohxstatus_count = data_college.groupby("college").agg("count")
college_ohxstatus = round(ohxstatus_college.div(ohxstatus_count, level="college")*100,3)
college_ohxstatus = college_ohxstatus.take([5], axis=1)
data_college_ohx = pd.concat([ohxstatus_college, college_ohxstatus], axis=1)
data_college_ohx.columns = ["count", "ratio"]
data_college_ohx["count and ratio"] = data_college_ohx.apply(lambda x: str(x["count"])+"("+str(x["ratio"])+"%)", axis=1)
res_college_ohx = data_college_ohx.take([2], axis=1)
# Chi-squared test
df = data_college_ohx.take([0], axis=1).values.transpose()
df_college = np.vstack(np.hsplit(df,2))
kf_college = chi2_contingency(df_college)
print('chisq-statistic=%.4f, p-value=%.50f, df=%i expected_frep=%s'%kf_college)
res_college_ohx["p value"] = kf_college[1]

chisq-statistic=215.9324, p-value=0.00000000000000000000000000000000000000000000000070, df=1 expected_frep=[[23336.94376855  2064.05623145]
 [11023.05623145   974.94376855]]


In [262]:
## For gender
data_gender = data_c
# modify "ohx_status" as "complete" or "missing"
data_gender["ohx status"] = data_c.apply(lambda x: "complete" if x["ohx_status"]=="Complete" else "missing", axis=1)
ohxstatus_gender = data_gender.groupby(["gender","ohx status"]).agg({"ohx status":"count"})
ohxstatus_count = data_gender.groupby("gender").agg("count")
gender_ohxstatus = round(ohxstatus_gender.div(ohxstatus_count, level="gender")*100,3)
gender_ohxstatus = gender_ohxstatus.take([5], axis=1)
gender_ohxstatus
data_gender_ohx = pd.concat([ohxstatus_gender, gender_ohxstatus], axis=1)
data_gender_ohx.columns = ["count", "ratio"]
data_gender_ohx["count and ratio"] = data_gender_ohx.apply(lambda x: str(x["count"])+"("+str(x["ratio"])+"%)", axis=1)
res_gender_ohx = data_gender_ohx.take([2], axis=1)
# Chi-squared test
df = data_gender_ohx.take([0], axis=1).values.transpose()
df_gender = np.vstack(np.hsplit(df,2))
kf_gender = chi2_contingency(df_gender)
print('chisq-statistic=%.4f, p-value=%.50f, df=%i expected_frep=%s'%kf_gender)
res_gender_ohx["p value"] = kf_gender[1]

chisq-statistic=10.1546, p-value=0.00143939250749849727804841048595108077279292047024, df=1 expected_frep=[[17426.68199684  1541.31800316]
 [16933.31800316  1497.68199684]]


In [263]:
# For under_20
# modify "age" into "Under 20" or "20 or older"
data_d = data_c
data_d["a"] = data_d.apply(lambda x: "Under 20" if x["age"]<20 else "20 or older", axis=1)
data_d = data_d[["id", "gender", "a", "college", "exam_status", "ohx_status", "cohort"]]
data_d = data_d.rename(columns={"a":"under_20"})
# modify "ohx_status" as "complete" or "missing"
data_d["ohx status"] = data_d.apply(lambda x: "complete" if x["ohx_status"]=="Complete" else "missing", axis=1)
ohxstatus_age = data_d.groupby(["under_20","ohx status"]).agg({"ohx status":"count"})
ohxstatus_count = data_d.groupby("under_20").agg("count")
age_ohxstatus = round(ohxstatus_age.div(ohxstatus_count, level="under_20")*100,3)
age_ohxstatus = age_ohxstatus.take([5], axis=1)
data_age_ohx = pd.concat([ohxstatus_age, age_ohxstatus], axis=1)
data_age_ohx.columns = ["count", "ratio"]
data_age_ohx["count and ratio"] = data_age_ohx.apply(lambda x: str(x["count"])+"("+str(x["ratio"])+"%)", axis=1)
res_age_ohx = data_age_ohx.take([2], axis=1)
#chi-squared test
df = data_age_ohx.take([0], axis=1).values.transpose()
df_age = np.vstack(np.hsplit(df,2))
kf_age = chi2_contingency(df_age)
print('chisq-statistic=%.4f, p-value=%.100f, df=%i expected_frep=%s'%kf_age)
res_age_ohx["p value"] = kf_age[1]

chisq-statistic=340.5023, p-value=0.0000000000000000000000000000000000000000000000000000000000000000000000000004959900703450187114134128, df=1 expected_frep=[[19887.07077729  1758.92922271]
 [14472.92922271  1280.07077729]]


In [264]:
data_age_continuous = data_c
# modify "ohx_status" as "complete" or "missing"
data_age_continuous["ohx status"] = data_c.apply(lambda x: "complete" if x["ohx_status"]=="Complete" else "missing", axis=1)
data_age_continuous["AGE"] = "age"
ohxstatus_age_c_mean = data_age_continuous.groupby(["AGE", "ohx status"]).mean()
ohxstatus_age_c_mean = ohxstatus_age_c_mean.take([1], axis=1)
ohxstatus_age_c_mean.columns = ["mean"]
ohxstatus_age_c_sd = data_age_continuous.groupby(["AGE", "ohx status"]).std()
ohxstatus_age_c_sd = ohxstatus_age_c_sd.take([1], axis=1)
ohxstatus_age_c_sd.columns = ["std"]
result_age = pd.concat([ohxstatus_age_c_mean, ohxstatus_age_c_sd], axis=1)
result_age["count and ratio"] = result_age.apply(lambda x: "mean="+str(x["mean"])+", std="+str(x["std"]), axis=1)
result_age = result_age[["count and ratio"]]
result_age

count and ratio
AGE ohx status                                                 
age complete      mean=33.16947031431898, std=24.36736205677066
    missing     mean=22.009542612701548, std=26.587235296888924

In [265]:
data_age_continuous = data_c
# modify "ohx_status" as "complete" or "missing"
data_age_continuous["ohx status"] = data_c.apply(lambda x: "complete" if x["ohx_status"]=="Complete" else "missing", axis=1)
# Separate age by ohx status(complete and missing)
complete_age = data_age_continuous.loc[data_age_continuous['ohx status']=='complete']
complete_ttest = list(complete_age["age"].values)
missing_age = data_age_continuous.loc[data_age_continuous['ohx status']=='missing']
missing_ttest = list(missing_age["age"].values)

result_age["p value"] = stats.ttest_ind(complete_ttest, missing_ttest).pvalue

result_age

count and ratio        p value
AGE ohx status                                                                
age complete      mean=33.16947031431898, std=24.36736205677066  1.770477e-126
    missing     mean=22.009542612701548, std=26.587235296888924  1.770477e-126

In [266]:
result_d = pd.concat([result_age, res_age_ohx, res_gender_ohx, res_college_ohx], 
                      keys=["age", "under_20", "gender", "college"],
                      axis=0)
result_d

count and ratio  \
                                       ohx status                                                    
age      age                           complete      mean=33.16947031431898, std=24.36736205677066   
                                       missing     mean=22.009542612701548, std=26.587235296888924   
under_20 20 or older                   complete                                   20369.0(94.101%)   
                                       missing                                      1277.0(5.899%)   
         Under 20                      complete                                   13991.0(88.815%)   
                                       missing                                     1762.0(11.185%)   
gender   Female                        complete                                   17342.0(91.428%)   
                                       missing                                      1626.0(8.572%)   
         Male                          complete                                   17018.0(92.334%)   
                                       missing                                      1413.0(7.666%)   
college  No college/<20                complete                                   22974.0(90.445%)   
                                       missing                                      2427.0(9.555%)   
         some college/college graduate complete                                   11386.0(94.899%)   
                                       missing                                       612.0(5.101%)   

                                                         p value  
                                       ohx status                 
age      age                           complete    1.770477e-126  
                                       missing     1.770477e-126  
under_20 20 or older                   complete     4.959901e-76  
                                       missing      4.959901e-76  
         Under 20                      complete     4.959901e-76  
                                       missing      4.959901e-76  
gender   Female                        complete     1.439393e-03  
                                       missing      1.439393e-03  
         Male                          complete     1.439393e-03  
                                       missing      1.439393e-03  
college  No college/<20                complete     6.977150e-49  
                                       missing      6.977150e-49  
         some college/college graduate complete     6.977150e-49  
                                       missing      6.977150e-49

In [267]:
# construct a table, include a caption: ---------------------------------------
cap = """
<b> Table 1.</b> <em> Age, under_20, gender and college with different OHX exam status</em>
For the rows corresponding to categorical variable in the table, each cell should provide a count (n) and a percent (of the row). 
For the continous variable age, report the mean and standard deviation [Mean (SD)] for each cell.
Include a column ‘p-value’ giving a p-value testing for a mean difference in age or an association 
between each categorical varaible and missingness. Use a chi-squared test comparing the 2 x 2 tables 
for each categorical characteristic and OHX exam status and a t-test for the difference in age.
"""
res = pd.DataFrame(result_d)
t1 = res.to_html(index=True, sparsify=False)
t1 = t1.rsplit('\n')
t1.insert(1, cap)
tab1 = ''
for i, line in enumerate(t1):
    tab1 += line
    if i < (len(t1) - 1):
        tab1 += '\n'
display(HTML(tab1))

,,,count and ratio,p value
,,ohx status,,
age,age,complete,"mean=33.16947031431898, std=24.36736205677066",1.770477e-126
age,age,missing,"mean=22.009542612701548, std=26.587235296888924",1.770477e-126
under_20,20 or older,complete,20369.0(94.101%),4.959901e-76
under_20,20 or older,missing,1277.0(5.899%),4.959901e-76
under_20,Under 20,complete,13991.0(88.815%),4.959901e-76
under_20,Under 20,missing,1762.0(11.185%),4.959901e-76
gender,Female,complete,17342.0(91.428%),1.439393e-03
gender,Female,missing,1626.0(8.572%),1.439393e-03
gender,Male,complete,17018.0(92.334%),1.439393e-03
